# Giới thiệu Convolution Nets

Convolutional Neural Networks (CNN) là một trong những mô hình deep learning phổ biến nhất và có ảnh hưởng nhiều nhất trong cộng đồng Computer Vision. CNN được dùng trong trong nhiều bài toán như nhân dạng ảnh, phân tích video, ảnh MRI, hoặc cho bài các bài của lĩnh vự xử lý ngôn ngữ tự nhiên,và hầu hết đều giải quyết tốt các bài toán này. 

CNN cũng có lịch sử khá lâu đời. Kiến trúc gốc của mô hình CNN được giới thiệu bởi một nhà khoa học máy tính người Nhật vào năm 1980. Sau đó, năm 1998, Yan LeCun lần đầu huấn luyện mô hình CNN với thuật toán backpropagation cho bài toán nhận dạng chữ viết tay. Tuy nhiên, mãi đến năm 2012, khi một nhà khoa học máy tính người Ukraine Alex Krizhevsky (đệ của Geoffrey Hinton) xây dựng mô hình CNN (AlexNet) và sử dụng GPU để tăng tốc quá trình huấn luyện deep nets để đạt được top 1 trong cuộc thi Computer Vision thường niên ImageNet với độ lỗi phân lớp top 5 giảm hơn 10% so với những mô hình truyền thống trước đó, đã tạo nên làn sóng mãnh mẽ sử dụng deep CNN với sự hỗ trợ của GPU để giải quyết càng nhiều các vấn đề trong Computer Vision.

# Bài Toán Phân loại Ảnh
Phân loại ảnh là một bài toán quan trọng bậc nhất trong lĩnh vực Computer Vision. Chúng ta đã có rất nhiều nghiên cứu để giải quyết bài toán này bằng cách rút trích các đặc trưng rất phổ biến như SIFT, HOG rồi cho máy tính học nhưng những cách này tỏ ra không thực sự hiểu quả. Nhưng ngược lại, đối với con người, chúng ta lại có bản năng tuyệt vời để phân loại được những đối tượng trong khung cảnh xung quanh một cách dễ làm.

Dữ liệu đầu vào của bài toán là một bức ảnh. Một ảnh được biểu ảnh bằng ma trận các giá trị. Mô hình phân lớp sẽ phải dự đoán được lớp của ảnh từ ma trận điểm ảnh này, ví dụ như ảnh đó là con mèo, chó, hay là chim.

![](https://pbcquoc.github.io/images/cnn_input.png)

# Nội dung 
Trong tut này, mình sẽ hướng dẫn các bạn xây dựng mô hình CNN (Convolution Neural Nets) cho bài toán phân loại ảnh. Các bạn sẽ sử dụng tensorflow [eager execution](https://www.tensorflow.org/guide/eager) để xây dựng model, huấn luyện mô hình trên tập train và predict ảnh trong tập test. 

Tut này sẽ có đi câú trúc như sau:
1. Import dữ liệu
2. Xây dựng mô hình
3. Huấn luyện mô hình
4. Đánh giá mô hình
5. Sử dụng mô hình đã huấn luyện để dự đoán

# Import thư viện

Chúng ta sử dụng một số hàm cơ bản trong tensorflow, sklearn và phải enable tf eage execution 

In [0]:
import os
import numpy as np
np.warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

from google_drive_downloader import GoogleDriveDownloader as gdd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.contrib.eager.python import tfe

tf.enable_eager_execution()
tf.set_random_seed(0)
np.random.seed(0)

# Import và inspect dữ liệu
Trong bài này, các bạn phải xây dựng mô hình để xác định các địa danh nổi tiếng trên lãnh thổ Việt Nam được mô tả trong bức ảnh. Tập dữ liệu huấn luyện bao gồm 20k ảnh, là một phần nhỏ của bộ dữ liệu trong cuộc thi ZaloAI năm 2018. 

Hình dưới mình họa một số địa danh nổi tiếng ở Việt Nam: Chùa một cột, vịnh hạ long
![](https://github.com/pbcquoc/cnn/raw/master/img/smaple.png)


## Download dữ liệu
Dự liễu đã được tổ chức và chuyển thành dạng ma trận sẵn, các bạn chỉ việc tải về mà không cần phải load từ file raw

In [0]:
gdd.download_file_from_google_drive(file_id='1ABhwNb5ioRzUEV9iLDpVSEIV_76yofNm', dest_path='./zaloai_landmark_20k.npz', unzip=False)

In [0]:
data = np.load("zaloai_landmark_20k.npz")
X, y = data['X'], data['y']

num_classes = len(np.unique(y))
y_ohe = tf.keras.utils.to_categorical(y, num_classes=num_classes, dtype='int')

## Chia dự liệu để huấn luyện và đánh giá
Chúng ta sử dụng hàm train_test_split trong thư viện sklearn để chia tập dữ liệu thành 2 phần train/test một cách nhanh chóng.

In [0]:
x_train, x_test, y_train_ohe, y_test_ohe = train_test_split(X, y_ohe, test_size=0.25)
print("Train size: {} - Test size: {}".format(x_train.shape, x_test.shape))

## Mô Hình CNN

CNN bao gồm tập hợp các lớp cơ bản bao gồm: convolution layer + nonlinear layer, pooling layer, fully connected layer. Các lớp này liên kết với nhau theo một thứ tự nhất định. Thông thường, một ảnh sẽ được lan truyền qua tầng convolution layer + nonlinear layer đầu tiên, sau đó các giá trị tính toán được sẽ lan truyền qua pooling layer, bộ ba convolution layer + nonlinear layer + pooling layer có thể được lặp lại nhiều lần trong network. Và sau đó được lan truyền qua tầng fully connected layer và softmax để tính sác xuất ảnh đó chứa vật thế gì.

![](https://pbcquoc.github.io/images/cnn_model.png)

### Convolution Layer
Convolution layer là lớp quan trọng nhất và cũng là lớp đầu tiên của của mô hình CNN. Lớp này có chức năng chính là phát hiện các đặc trưng có tính không gian hiệu quả. Trong tầng này có 4 đối tượng chính là: ma trận đầu vào, bộ **filters**, và **receptive field**, **feature map**. Conv layer nhận đầu vào là một ma trận 3 chiều và một bộ filters cần phải học. Bộ filters này sẽ trượt qua từng vị trí trên bức ảnh để tính tích chập (convolution) giữa bộ filter và phần tương ứng trên bức ảnh. Phần tưng ứng này trên bức ảnh gọi là receptive field, tức là vùng mà một neuron có thể nhìn thấy để đưa ra quyết định, và mà trận cho ra bới quá trình này được gọi là feature map. Để hình dung, các bạn có thể tưởng tượng, bộ filters giống như các tháp canh trong nhà tù quét lần lượt qua không gian xung quanh để tìm kiếm tên tù nhân bỏ trốn. Khi phát hiện tên tù nhân bỏ trốn, thì chuông báo động sẽ reo lên, giống như các bộ filters tìm kiếm được đặc trưng nhất định thì tích chập đó sẽ cho giá trị lớn. 

<div class="img-div" markdown="0">
    <img src="https://media.giphy.com/media/3orif7it9f4phjv4LS/giphy.gif" />
</div>

Với ví dụ ở bên dưới, dữ liệu đầu vào ở là ma trận có kích thước 8x8x1, một bộ filter có kích thước 2x2x1, feature map có kích thước 7x7x1. Mỗi giá trị ở feature map được tính bằng tổng của tích các phần tử tương ứng của bộ filter 2x2x1 với receptive field trên ảnh. Và để tính tất cả các giá trị cho feature map, các bạn cần trượt filter từ trái sáng phải, từ trên xuống dưới. Do đó, các bạn có thể thấy rằng phép convolution bảo toàn thứ tự không gian của các điểm ảnh. ví dụ điểm góc gái của dữ liệu đầu vào sẽ tương ứng với bên một điểm bên góc trái của feature map. 

<div class="img-div" markdown="0">
    <img src="https://pbcquoc.github.io/images/cnn_covolution_layer.png" />
</div>

#### Tầng convolution như là feature detector 

Tầng convolution có chức năng chính là phát hiện đặc trưng cụ thể của bức ảnh. Những đặc trưng này bao gồm đặc trưng cơ bản là góc,cạnh, màu sắc, hoặc đặc trưng phức tạp hơn như texture của ảnh. Vì bộ filter quét qua toàn bộ bức ảnh, nên những đặc trưng này có thể nằm ở vị trí bất kì trong bức ảnh, cho dù ảnh bị xoáy trái/phải thì những đặc trưng này vẫn bị phát hiện. 

Ở minh họa dưới, các bạn có một filter 5x5 dùng để phát hiện góc/cạnh với, filter này chỉ có giá trị một tại các điểm tương ứng một góc cong. 

<div class="img-div" markdown="0">
    <img src="https://pbcquoc.github.io/images/cnn_high_level_feature.png" />
</div>

Dùng filter ở trên trược qua ảnh của nhân vật Olaf trong trong bộ phim Frozen. Chúng ta thấy rằng, chỉ ở những vị trí trên bức ảnh có dạng góc như đặc trưng ở filter thì mới có giá trị lớn trên feature map, những vị trí còn lại sẽ cho giá trị thấp hơn. Điều này có nghĩa là, filter đã phát hiện thành công một dạng góc/cạnh trên dự liệu đầu vào. Tập hơn nhiều bộ filters sẽ cho phép các bạn phát hiện được nhiều loại đặc trưng khác nhau,và giúp định danh được đối tượng. 

<div class="img-div" markdown="0">
    <img src="https://pbcquoc.github.io/images/cnn_high_level_feature_ex.png" />
</div>

#### Các tham số của tầng convolution: Kích thước bộ filter, stride và padding

Kích thước bộ filter là một trong những tham số quan trọng nhất của tầng convolution. Kích thước này tỉ lệ thuận với số tham số cần học tại mỗi tầng convolution và là tham số quyết định receptive field của tầng này. Kích thước phổ biến nhất của bộ filter là 3x3.


# Xây dựng mô hình
Các bạn cần phải xây dựng mô hình CNN có kiến trúc sau đây. Các bạn phải thỏa mãn bộ filter có kích thước 3x3. Đối với các tham số còn lại, các bạn có thể tự do lựa chọn để cho ra kết quả huấn luyện tốt nhất.

![](https://github.com/pbcquoc/cnn/raw/master/images/cnn_architecture_2.png)


## Định nghĩa block CNN
Để hỗ trợ quá trình định nghĩa mô hình. Các bạn cần định nghĩa một block bao gồm 3 lớp sau: Conv2D, MaxPool2D, ReLU. Block này sẽ được tái sử dụng nhiều lần trọng networks. Các layers cần được khai báo trong hàm init và được gọi trong hàm call. Hãy tham khảo ví dụ dưới đây.

```python

class ConvBlock(tf.keras.Model):
    def __init__(self):
        super(ConvBlock, self).__init__()
        self.cnn = tf.keras.layers.Conv2D(32, (3, 3), strides=(1, 1),  padding="same")
        
    def call(self, inputs, training=None, mask=None):
        x = self.cnn(inputs)

        return x
```

In [0]:
class ConvBlock(tf.keras.Model):
    def __init__(self, filters, kernel, strides, padding):
        super(ConvBlock, self).__init__()
        ## TODO 1 ##
        
        ## END TODO 1 ##
        self.cnn = tf.keras.layers.Conv2D(filters, (kernel, kernel), strides=(strides, strides), kernel_initializer='he_normal', padding=padding)
        self.pool = tf.keras.layers.MaxPool2D((2,2), strides=(2,2))
        self.bn = tf.keras.layers.BatchNormalization()
        
    def call(self, inputs, training=None, mask=None):
        ## TODO 2 ##
        
        ## END TODO 2 ##
        x = self.cnn(inputs)
        x = self.bn(x)
        x = tf.nn.relu(x)   
        x = self.pool(x)

        return x

## Định nghĩa toàn bộ mô hình CNN
Các bạn sử dụng block ở trên để định nghĩa toàn bộ mô hình CNN có kiến trúc như hình dưới. Các layer cần được khởi tạo trong hàm init, và được gọi trong hàm call.

In [0]:
class CNN(tf.keras.Model):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        
        ## TODO 3 ##
        ## END TODO 3 ##
        self.block1 = ConvBlock(64, kernel=3, strides=1, padding='same')
        self.block2 = ConvBlock(128, kernel=3, strides=1, padding='same')
        self.block3 = ConvBlock(256, kernel=3, strides=1, padding='same')
        self.block4 = ConvBlock(512, kernel=3, strides=1, padding='same')
        self.block5 = ConvBlock(512, kernel=3, strides=1, padding='same')
        self.block6 = ConvBlock(1024, kernel=3, strides=1, padding='same')
        self.flatten = tf.layers.Flatten()
        
        ## TODO 4 ##
        ## END TODO 4 ##
        self.dense2 = tf.keras.layers.Dense(num_classes)

    def call(self, inputs, training=None, mask=None):
        
        ## TODO 5 ##
        ## END TODO 5 ##
        x = self.block1(inputs)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)
        x = self.block6(x)
        
        x = self.flatten(x)
        
        ## TODO 6 ##
        ## END TODO 6 ##   
        x = self.dense2(x)
        
        # softmax op does not exist on the gpu, so always use cpu
        with tf.device('/cpu:0'):
            output = tf.nn.softmax(x)

        return output

# Huấn Luyện
Đoạn code này thực hiện quá trình huấn luyện mô hình CNN. Mỗi lần chạy mô hình sẽ lấy batch_size mẫu dữ liệu, feedforward, tính loss, và cập nhật gradient cho toàn bộ trọng số. Toàn bộ quá trình này được thực hiện trong hàm fit()

Sau khi huấn luyện xong, chúng ta sẽ đánh giá độ chính xác của mô hình bằng hàm evaluate().

In [0]:
device = '/cpu:0' if tfe.num_gpus() == 0 else '/gpu:0'
batch_size = 32
epochs = 16

with tf.device(device):
    # build model and optimizer
    model = CNN(num_classes)
    model.compile(optimizer=tf.train.AdamOptimizer(0.001), loss='categorical_crossentropy',
                  metrics=['accuracy'])
    # TF Keras tries to use entire dataset to determine shape without this step when using .fit()
    # Fix = Use exactly one sample from the provided input dataset to determine input/output shape/s for the model
    
#     dummy_x = tf.zeros((1, 224, 224, 3))
#     model._set_inputs(dummy_x)
#     model.summary()

    # train
    model.fit(x_train, y_train_ohe, batch_size=batch_size, epochs=epochs,
              validation_data=(x_test, y_test_ohe), verbose=1)

    # evaluate on test set
    scores = model.evaluate(x_test, y_test_ohe, batch_size, verbose=1)
    print("Final test loss and accuracy :", scores)

    model.save_weights('./check_points/my_model')


# Dự Đoán 

Chúng ta sử dụng mô hình đã được huấn luyện bên trên để dự đoán cho một ảnh bất kì. Đầu tiên là load lại bộ trọng số đã được huấn luyện. Sau đó, sử dụng mô hình này để dự đoán cho các ảnh mới. 

In [0]:
model = CNN(num_classes)
model.load_weights('./check_points/my_model')
print("Model đã được load")

In [0]:
x_new = x_test[0]
pred = model.predict(x_new[None, :])
pred_label = np.argmax(pred)
plt.imshow(x_new)
print("Mô hình dự đoán nhãn của bức ảnh: {}".format(pred_label))